In [ ]:
# 파일 불러오기
import pandas as pd
train=pd.read_csv("test_data.csv")

In [ ]:
# 토픽별 도수분포표 확인
import matplotlib.pyplot as plt
train_new=train.groupby(["topic_idx"]).count()
plt.bar(train_new.index,train_new["index"])
plt.show()

In [ ]:
# 뉴스 기사 불용어 확인
b=list()
c=list()
for i in range(train.shape[0]) :
    a=train.loc[i]["title"]
    a=a.split()
    b.append(a[-1])
    c.append(a[0])

a=pd.DataFrame()
a["end"]=b
a["start"]=c

In [ ]:
a1=a.groupby(a["start"]).count()
a1=a1.sort_values(ascending=False, by="end")
a1.head(50)

In [ ]:
a2=a.groupby(a["end"]).count()
a2=a2.sort_values(ascending=False, by="start")
a2.head(50)

In [ ]:
# 전처리
def word_delete(title):
    delete_word = [
        "1보",
        "2보",
        "3보",
        "4보",
        "5보",
        "6보",
        "게시판",
        "신간",
        "그래픽",
        "속보",
        "영상",
        "...",
        "주말 N",
        "QA",
        "동정",
        "위클리",
        "주간 화제의 뉴스",
        "카드뉴스",
        "팩트체크",
######## 스포츠 10대 뉴스, 관련주 등 키워드 추론 가능 내용은 제거하지 않음
    ]
    for i in delete_word:
        if title.endswith(i) or title.startswith(i):
            title = title.replace(i, "")
    if title.endswith("종합"):
        title = title[:-2]
    title = title.replace("↑", " 증가")
    title = title.replace("↓", " 감소")
    title = title.replace("→", "에서 ")
    title = title.replace("~", "에서 ")
    title = title.replace("·", " 그리고 ")
    title = title.replace(":", " 대 ")
    title = title.replace("%", "퍼센트")
    title=title.replace("…"," ")
    title=title.replace("...", " ")
    title=title.replace("&","그리고")
    return title

# 빈도가 많은 용어들 중 토픽과 상관없는 불용어 제거
# 빈도가 적어도 위클리 뉴스 등 필요없는 불용어 제거
# 어미에 종합이 있는 경우 및 각종 기호 등 제거
# %, …가 남아있을 경우 번역 시 결측치 발생하여 퍼센트로 변경 후 사용

In [ ]:
def word_eda(title) :
    title=word_delete(title)
    return title

train['title']=train['title'].apply(word_eda)

In [ ]:
# 번역
!pip install tqdm

In [ ]:
import sys
import numpy as np
import random
from tqdm.notebook import tqdm
import os
import re
import time
from collections import Counter

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.1 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,136 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/

In [ ]:
path=os.getcwd()
path

'/content'

In [ ]:
data_path='\content'

In [ ]:
def chrome_setting():
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome(options=chrome_options)
  return driver

driver=chrome_setting()

In [ ]:
# Crawling 한->영 번역
def kor_to_trans(text_data, trans_lang,start_index,final_index):

  target_present = EC.presence_of_element_located((By.XPATH, '//*[@id="txtTarget"]'))

  for i in tqdm(range(start_index,final_index)):

    if (i!=0)&(i%99==0):
      time.sleep(0.05)
      np.save(data_path+'kor_to_eng_train_{}_{}.npy'.format(start_index,final_index),trans_list)
    elif i==(final_index) :
      time.sleep(0.05)
      np.save(data_path+'kor_to_eng_train_{}_{}.npy'.format(start_index,final_index),trans_list)

    try:
      driver.get('https://papago.naver.com/?sk=ko&tk='+trans_lang+'&st='+text_data[i])
      time.sleep(1.5)
      element=WebDriverWait(driver, 10).until(target_present)
      time.sleep(0.1)
      backtrans = element.text

      if (backtrans=='')|(backtrans==' '):
        element=WebDriverWait(driver, 20).until(target_present)
        backtrans = element.text
        trans_list.append(backtrans)
      else:
        trans_list.append(backtrans)

    except:
      trans_list.append('')

In [ ]:
a=train.shape[0]
trans_list=[]
kor_to_trans(train['title'], 'en',0,a)
np.save(data_path+'kor_to_eng_train.npy',trans_list)
# 실제 데이터는 너무 커서 10000개씩 구간을 나눠서 번역 후 결합
# time sleep이나 wait time이 짧아 결측치가 생기는 문제가 발생하여 그 시간을 늘림

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
data2=np.load('\contentkor_to_eng_train.npy')
train_eng=pd.DataFrame()
train_eng["title_en"]=data2

In [ ]:
train_eng["index"]=train["index"]
train_eng["title"]=train["title"]
train_eng["topic_idx"]=train["topic_idx"]

In [ ]:
train_eng[trian_eng["title_en"].isnull()]
# 이거 아니면 train_eng[train_eng["title_en"]==''] 사용
# 결측치가 있을 시 새로운 데이터로 설정하여 재분석 실시 후 concat함수를 통해 결합

In [ ]:
## 결측치 재번역
train_eng_null=train_eng[trian_eng["title_en"]=='']
train_eng_null=train_eng_null.reset_index(drop=True)
a=train_eng_null.shape[0]
trans_list=[]
kor_to_trans(train_eng_null['title'], 'en',0,a)
np.save(data_path+'kor_to_eng_null_train.npy',trans_list)

In [ ]:
data2=np.load('Untitled Folderkor_to_eng_null_train.npy')
train_eng_null=pd.DataFrame()
train_eng_null["index"]=train_null["index"]
train_eng_null["title"]=train_null["title"]
train_eng_null["title_en"]=data2

In [ ]:
# 인덱스 재설정
train_eng["index2"]=train_eng["index"]
train_eng=train_eng_null.set_index("index2")
train_eng_null["index2"]=train_eng["index"]
train_eng_null=train_eng_null.set_index("index2")

In [ ]:
id=train_eng_null.index
train_eng2=train_eng.drop(id)
train_eng=pd.concat([train_eng_null,train_eng])
train_eng=train_eng.sort_values(by="index")
train_eng=train_eng.reset_index(drop=True)

In [ ]:
train_eng.to_csv("train_eng.csv")

In [ ]:
def per_eda(title) :
    title=title.replace("%","percent")
    title=title.replace("...", " ")
    title=title.replace("…"," ")
    title=title.replace("&","end")
    return title

train_eng=pd.read_csv("train_eng.csv") # 한글이 안될 시 encoding='cp949' 사용
train_eng['title_en']=train_eng['title_en'].apply(per_eda)

## 퍼센트가 영어 번역 시 다시 %로 번역되는 등 특수문자가 생기는 경우가 생겨 변환 후 번역 실시

In [ ]:
def en_to_trans(text_data, trans_lang,start_index,final_index):

  target_present = EC.presence_of_element_located((By.XPATH, '//*[@id="txtTarget"]'))

  for i in tqdm(range(start_index,final_index)):

    if (i!=0)&(i%99==0):
      time.sleep(0.05)
      np.save(data_path+'eng_to_kor_train_{}_{}.npy'.format(start_index,final_index),trans_list)
    elif i==final_index :
      time.sleep(0.05)
      np.save(data_path+'eng_to_kor_train_{}_{}.npy'.format(start_index,final_index),trans_list)

    try:
      driver.get('https://papago.naver.com/?sk=en&tk='+trans_lang+'&st='+text_data[i])
      time.sleep(1.5)
      element=WebDriverWait(driver, 10).until(target_present)
      time.sleep(0.1)
      backtrans = element.text

      if (backtrans=='')|(backtrans==' '):
        element=WebDriverWait(driver, 20).until(target_present)
        backtrans = element.text
        trans_list.append(backtrans)
      else:
        trans_list.append(backtrans)

    except:
      trans_list.append('')

In [ ]:
a=train_eng.shape[0]
trans_list=[]
en_to_trans(train_eng['title_en'], 'ko',0,a)
np.save(data_path+'eng_to_kor_train.npy',trans_list)

In [ ]:
data2=np.load('Untitled Foldereng_to_kor_train.npy')
train_eng["title_kor"]=data2
## 이 과정에서도 결측치 생길 시 한-영과 같은 과정 거쳐서 처리

In [ ]:
def ch_eda(title) :
    title=title.replace("英","Great Britain")
    title=title.replace("美", "the United States")
    title=title.replace("伊","Italy")
    title=title.replace("與","the rulling party")
    title=title.replace("野","the opposition party")
    title=title.replace("獨","Germany")
    title=title.replace("女","woman")
    title=title.replace("親","friendly")
    title=title.replace("亞","Asia")
    title=title.replace("反","opposing")
    title=title.replace("印","India")
    title=title.replace("檢","the prosecution")
    title=title.replace("佛","France")
    return title

In [ ]:
# 영-한 역번역 시 한자의 경우 제대로 번역이 되지 않는 문제가 발생
# 한자가 포함된 데이터 확인 후 ch_eda 적용 후 재번역 실시
def contains_hanja(text):
  hanja_range = (0x4E00, 0x9FFF)  # 한자 유니코드 범위
  for char in text:
    if hanja_range[0] <= ord(char) <= hanja_range[1]:
      return True
    else:
      return False

kor_data2_hanja = train_eng[train_eng['title_en'].apply(contains_hanja)]

In [ ]:
# 한자어 포함 데이터프레임 추출
example_list=["英","美","伊","獨","野","與","女","親","亞","反","印","檢","佛"]
test = '|'.join(example_list)
train_eng_c=train_eng[train_eng["title_en"].str.contains(test)]

In [ ]:
train_eng_c=train_eng_c.reset_index(drop=True)
train_eng_c["title_en"]=train_eng_c["title_en"].apply(ch_eda)

In [ ]:
a=train_eng_c.shape[0]
trans_list=[]
en_to_trans(train_eng_c['title_en'], 'ko',0,a)
np.save(data_path+'eng_to_kor_train_c.npy',trans_list)

In [ ]:
data2=np.load('Untitled Foldereng_to_kor_train_c.npy')
train_eng_ckor=pd.DataFrame()
train_eng_ckor["index"]=train_eng_c["index"]
train_eng_ckor["topic_idx"]=train_eng_c["topic_idx"]
train_eng_ckor["title"]=train_eng_c["title"]
train_eng_ckor["title_en"]=train_eng_c["title_en"]
train_eng_ckor["title_kor"]=data2

In [ ]:
# 한자어 적용 시 생기는 결측치 제거 및 인덱스 설정
train_eng_ckor=train_eng_ckor.dropna()
train_eng_ckor["index2"]=train_eng_ckor["index"]
train_eng_ckor=train_eng_ckor.set_index("index2")

In [ ]:
train_f=pd.DataFrame()
train_f["index"]=train_eng["index"]
train_f["topic_idx"]=train_eng["topic_idx"]
train_f["title"]=train_eng["title"]
train_f["title_en"]=train_eng["title_en"]
train_f["title_kor"]=train_eng["title_kor"]

In [ ]:
train_f=train_f.sort_values(by="index")
train_f=train_f.reset_index(drop=True)

In [ ]:
### train_eng_ckor에 해당하는 index 제거
id=train_eng_ckor.index
train_fin=train_fi.drop(id)
train_fina=pd.concat([train_fin,train_eng_ckor])

In [ ]:
train_final=pd.DataFrame()
train_final["index"]=train_fina["index"]
train_final["title"]=train_fina["title"]
train_final["topic_idx"]=train_fina["topic_idx"]
train_final["title_en"]=train_fina["title_en"]
train_final["title_kor"]=train_fina["title_kor"]

In [ ]:
train_final=train_final.sort_values(by="index")
train_final=train_final.reset_index(drop=True)
train_final.to_csv("train_final.csv")

In [ ]:
import pandas as pd
train_final=pd.read_csv("train_final.csv")

In [ ]:
def ch2_eda(title) :
    title=title.replace("英","영국")
    title=title.replace("美", "미국")
    title=title.replace("伊","이탈리아")
    title=title.replace("與","여당")
    title=title.replace("野","야당")
    title=title.replace("獨","독일")
    title=title.replace("女","여자")
    title=title.replace("親","친")
    title=title.replace("亞","아시아")
    title=title.replace("反","반")
    title=title.replace("印","인도")
    title=title.replace("檢","검찰")
    title=title.replace("佛","프랑스")
    title=title.replace("靑", "청와대")
    title=title.replace("北", "북한")
    title=title.replace("韓", "한국")
    title=title.replace("日", "일본")
    title=title.replace("中", "중국")
    return title

In [ ]:
def per2_eda(title) :
    title=title.replace("~", "에서 ")
    title=title.replace("·", " 그리고 ")
    title=title.replace(":", " 대 ")
    title=title.replace("%","퍼센트")
    title=title.replace("...", " ")
    title=title.replace("…"," ")
    title=title.replace("&","그리고")
    return title

In [ ]:
### 역번역 자료에서 한자 및 특정 문자 변환
train_final["title_kor"]=train_final["title_kor"].apply(ch2_eda)
train_final["title_kor"]=train_final["title_kor"].apply(per2_eda)

In [ ]:
### 특수문자, 한자 등 없애기
import re

def remove_chr(str):

    new = re.sub(r'[^a-zA-Z가-힣0-9\s]', '', str) #
    return new

# train_final 데이터에 적용
train_final1 = train_final
temp = train_final1['title_kor'].apply(remove_chr)
temp2 = train_final1['title_en'].apply(remove_chr)
temp3 = train_final1['title'].apply(remove_chr)
train_final1['title'] = temp3
train_final1['title_kor'] = temp
train_final1['title_en'] = temp2

train_final1